# Scraping of features and values in the final link

This 

In [1]:
from random import sample 
import sqlite3 as sql
import pandas as pd


from bs4 import BeautifulSoup
import math
import tqdm
import json
import pandas as pd
import datetime
import time

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from IPython.core.debugger import Tracer
import time
from random import sample 

In [2]:
class UsedCars_Downloader:  
    '''
    UsedCars_Downloader was created to scrape all the sub-links of advertisements of used cars offered via platform 
    https://www.autoscout24.com/ and extract the various details about 
    random selected vehicles. 
    For the currect porposes the Downloader was set to download the vehicles of the most famous brands offered in Austria.
    (Note that scraper alover a user to change this setting)
    '''
    def getSoup(self, link):
        r = requests.get(link)
        r.encoding = 'UTF-8'
        return BeautifulSoup(r.text,'lxml')

    def getAllValues(self, link):
        '''
        getAllValues scrapes the information regarding the features get by getAllFeatures 
        (for instance the value of technical features, colour etc.)  
        '''
        soup = self.getSoup(link)
        qs= soup.findAll('dd')
        return [q.text[1:-1] for q in qs]

    def getAllFeatures(self, link):
        '''
        getAllFeature crapes the available variables names available for the selected cars. getAllValues than scrape the exact values
        '''
        soup = self.getSoup(link)
        zs = soup.findAll('dt')
        return [z.text for z in zs]



    def getTitle(self, link):
        '''
        getTitle scrapes the name of the selected advertisement that in many cases refere to the name of a vehicle and
        the most important categories that should attract a potentional buyer
        '''
        soup = self.getSoup(link)
        return soup.find('h2').text



    def getPrice(self, link):
        '''
        getPrice scrapes the ask price of a vehicle
        '''
        soup = self.getSoup(link)
        return soup.find('div', {"class":"cldt-price"}).find('h2').text[1:-1]



    def getPrimaryspecs(self, link):
        '''
        getPrimaryspecs scrapes primary specification of a car such as mileage, age etc. 
        '''
        soup = self.getSoup(link)
        a = soup.find('div', {"class":"cldt-stage-basic-data"}).findAll('div')[0].find('span').text
        b = soup.find('div', {"class":"cldt-stage-basic-data"}).findAll('div')[1].find('span').text
        c = soup.find('div', {"class":"cldt-stage-basic-data"}).findAll('div')[2].find('span').text
        d = soup.find('div', {"class":"cldt-stage-basic-data"}).findAll('div')[3].find('span').text


        return a,b,c,d 

        
    def getPrimaryspecs2(self, link):
        '''
        getPrimaryspecs scrapes the rest of primary specification of a vehicle
        '''
        soup = self.getSoup(link)
        extract = soup.find('div', {"class":"cldt-stage-basic-data"}).findAll('div')[3]
        xs = extract.findAll('span')
        return [x.text for x in xs]

    def RandomSample(self, df, coef):
        '''
        RandomSample creates a random selection within given data and selected amount of outputs
        '''
        x=sample(df, coef)
        return x
    
    def checker2(self, df):  
        l=[]
        for link in df:
            checker=requests.get(link)
            if str(checker) == '<Response [410]>':
                check = 'False'
            if str(checker) == '<Response [200]>':
                check = 'True'
            d = {
                'link':link,
                'Response':check,
        
            
                }
            l.append(d)
        D=pd.DataFrame(l)
        exists =  D['Response']=='True'
        DF=D[exists]
        DF= DF['link'].values.tolist()
        return DF
            
        
    def scraper(self, links, nmbroflinks, grouplen):
        start= time.time()
        tracker=0

        database = 'info13.1.2020.db'
        connection = sql.connect(database)
        l = []
        for link in links[0:nmbroflinks]:
            Features=self.getAllFeatures(link)
            Values=self.getAllValues(link)
            Title=self.getTitle(link)
            Price=self.getPrice(link) 
            Primaryspecs=self.getPrimaryspecs(link)
            Primaryspecs2=self.getPrimaryspecs2(link) 
            d = {
            'link':link,
            'Title':Title,
            'Price':Price,
            'km':Primaryspecs[0],
            'Zustand':Primaryspecs[1],
            'Registration date':Primaryspecs[2],
            'kW':Primaryspecs[3],
            'kW':Primaryspecs2[0],
            'hp': Primaryspecs2[1],
            'transmission': Primaryspecs2[2],
            'Ffuel': Primaryspecs2[2],
            Features[0]:Values[0],
            Features[1]:Values[1],
            Features[2]:Values[2],
            Features[3]:Values[3],
            Features[4]:Values[4],
            Features[5]:Values[5]
            }
            try:
                d[Features[6]] = Values[6]
            except:
                pass
            try:
                d[Features[7]] = Values[7]
            except:
                pass
            try:
                d[Features[8]] = Values[8]
            except:
                pass
            try:
                d[Features[9]] = Values[9]
            except:
                pass
            try:
                d[Features[10]] = Values[10]
            except:
                pass
            try:
                d[Features[11]] = Values[11]
            except:
                pass
            try:
                d[Features[12]] = Values[12]
            except:
                pass
            try:
                d[Features[13]] = Values[13]
            except:
                pass
            try:
                d[Features[14]] = Values[14]
            except:
                pass
            try:
                d[Features[15]] = Values[15]
            except:
                pass
            try:
                d[Features[16]] = Values[16]
            except:
                pass
            try:
                d[Features[17]] = Values[17]
            except:
                pass
            try:
                d[Features[18]] = Values[18]
            except:
                pass
            try:
                d[Features[19]] = Values[19]
            except:
                pass
            try:
                d[Features[20]] = Values[20]
            except:
                pass
            try:
                d[Features[21]] = Values[21]
            except:
                pass
            try:
                d[Features[22]] = Values[22]
            except:
                pass
            
            l.append(d)
            tracker = tracker +1

            if tracker % grouplen == 0:
                
                info1 =pd.DataFrame(l)
                try:
                    info1.to_sql('infotest2', connection, if_exists='append')
                except:
                    data=pd.read_sql_query('''SELECT * from infotest2''', connection)
                    df2=pd.concat([data, info1])
                    df2.to_sql('infotest2', connection, if_exists='replace')
                l = []
            time.sleep(1)



        print(tracker)           
        end=time.time()            
        print(end - start)
        print(info1)


# Data inputs

In [3]:
database = 'Austriandatabase.db'
connection = sql.connect(database)

df = pd.read_sql_query('''SELECT * from Austrianautolinksupto100k''', connection)
df= df['links'].values.tolist()

df

['https://www.autoscout24.com//offers/fiat-cinquecento-normal-gasoline-white-7d414899-4c62-df63-e053-e250040af717',
 'https://www.autoscout24.com//offers/kia-stonic-silber-1-4-mpi-mt6-100-isg-gasoline-red-d3e888ea-d999-4cf4-bfe8-cb0450682584',
 'https://www.autoscout24.com//offers/volkswagen-golf-tdi-diesel-black-c715889b-a3ff-b028-e053-e250040af7f5',
 'https://www.autoscout24.com//offers/mazda-323-p-1-3i-gasoline-silver-c9a2ac99-2aa7-5819-e053-e250040a4da6',
 'https://www.autoscout24.com//offers/peugeot-106-husky-gasoline-red-71ca8696-907b-ef6c-e053-e250040ab07f',
 'https://www.autoscout24.com//offers/audi-a6-2-5-tdi-diesel-green-4e114498-e189-5405-e053-e250040ad230',
 'https://www.autoscout24.com//offers/audi-100-2-3-e-gasoline-silver-1f86108f-2b66-2178-e053-e250040aa468',
 'https://www.autoscout24.com//offers/ford-focus-futura-gasoline-brown-4a11808e-74d2-5141-e053-e250040a6946',
 'https://www.autoscout24.com//offers/mercedes-benz-c-220-t-elegance-cdi-diesel-grey-7e113099-dbce-d509-

## Random Sampling and Cleaning of the Nonexisting Sub-links 

Random Sampling: According to the needs of the user of the code, the Random Sample that will be scraped can be changed by the coef value.

Nonexisting Sub-links: The Carlinks-Scraper takes long time and between the start of the scraping of sublinks and its end could some of those links be deleted. For this reason it is necessary to run the following information-scraping functions only on the existing sub-links. Checker2 can do this procedure. 

In [4]:
usedcars=UsedCars_Downloader()

In [5]:
coef=20

In [6]:
DF=usedcars.checker2(usedcars.RandomSample(df, coef))
len(DF)
DF

['https://www.autoscout24.com//offers/audi-s8-5-2-v10-quattro-facelift-ceramic-massage-randvoll-gasoline-silver-db452901-20b7-405c-8eaa-23c4add6d0af',
 'https://www.autoscout24.com//offers/volkswagen-t4-california-exclusive-2-5tdi-original-diesel-white-246bfbf6-3617-45ee-80cf-ae2ef56bbe21',
 'https://www.autoscout24.com//offers/mazda-cx-5-g194-awd-revolution-top-sd-leder-weiss-aut-gasoline-grey-c0350f59-cdf1-4e51-a516-61c12919ed6e',
 'https://www.autoscout24.com//offers/fiat-tipo-1-4-95-easy-gasoline-black-29c8e095-8f72-4acf-8dff-fbfa5faa8d3c',
 'https://www.autoscout24.com//offers/audi-a4-avant-1-8-t-fsi-gasoline-grey-8f3d409b-cec2-5533-e053-e250040a6531',
 'https://www.autoscout24.com//offers/bmw-x5-3-0sd-diesel-grey-26ca329c-3765-d86b-e053-e250040a995d',
 'https://www.autoscout24.com//offers/ford-focus-traveller-2-0-ecoblue-scr-st-line-aut-led-schei-diesel-black-e9fa697c-442d-4373-a8c6-bf67a514838b',
 'https://www.autoscout24.com//offers/ford-focus-traveller-1-0-ecoboost-trend-gasol

## Scraping

According to the needs of the user the nmbroflinks and grouplen can be changed. This part will ensure to optimize the equilibrium between the scraping speed and the semi-intervals of storing the data. 

Nmboflinks defines the final number of links that have to be scrapped.

In [7]:
nmbroflinks=len(DF)

Grouplen defines the number of links that are scraped within one loop.

In [8]:
len(DF)

19

In [9]:
grouplen=len(DF)

In [10]:
usedcars.scraper(DF, nmbroflinks, grouplen)

C:\Users\Radka\Anaconda3\lib\site-packages\pandas\core\generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


19
61.22995924949646
                                                 link  \
0   https://www.autoscout24.com//offers/audi-s8-5-...   
1   https://www.autoscout24.com//offers/volkswagen...   
2   https://www.autoscout24.com//offers/mazda-cx-5...   
3   https://www.autoscout24.com//offers/fiat-tipo-...   
4   https://www.autoscout24.com//offers/audi-a4-av...   
5   https://www.autoscout24.com//offers/bmw-x5-3-0...   
6   https://www.autoscout24.com//offers/ford-focus...   
7   https://www.autoscout24.com//offers/ford-focus...   
8   https://www.autoscout24.com//offers/mercedes-b...   
9   https://www.autoscout24.com//offers/volkswagen...   
10  https://www.autoscout24.com//offers/seat-ibiza...   
11  https://www.autoscout24.com//offers/skoda-octa...   
12  https://www.autoscout24.com//offers/peugeot-ri...   
13  https://www.autoscout24.com//offers/citroen-c3...   
14  https://www.autoscout24.com//offers/jeep-compa...   
15  https://www.autoscout24.com//offers/honda-acco...   
16  https:

[19 rows x 42 columns]


## Data cleaning

In [13]:
database = 'info13.1.2020.db'
connection = sql.connect(database)

In [14]:
info1 = pd.read_sql_query('''SELECT * from infotest2''', connection)

In [15]:
sep=','
for y in list(range(1,len(info1['transmission']))):
    transmission2=info1['transmission'][y].split(sep, 1)[0]
    fuel2=info1['transmission'][y].split(sep, 1)[1]

In [16]:
info1['transmission']=transmission2
info1['Ffuel']=fuel2

In [17]:
info1

,index,link,Title,Price,km,Zustand,Registration date,kW,hp,transmission,...,ConsumptionYou can obtain more information on the official fuel consumption and official specific CO2 emissions of new passenger vehicles from the guideline on fuel consumption and CO2 emissions of new passenger vehicles. This guideline is available free of charge at all dealerships and from Deutsche Automobil Treuhand GmbH at www.dat.de.,Next Inspection,Offer Number,Body Color Original,Warranty,CO2 EmissionYou can obtain more information on the official fuel consumption and official specific CO2 emissions of new passenger vehicles from the guideline on fuel consumption and CO2 emissions of new passenger vehicles. This guideline is available free of charge at all dealerships and from Deutsche Automobil Treuhand GmbH at www.dat.de.,Emission Class,Model Code,Emission Label,Availability
0,0,https://www.autoscout24.com//offers/audi-s8-5-...,Audi S8 5.2 V10 quattro *FACELIFT*CERAMIC*MASS...,"€ 9,900.-","300,000 km","Used, Full Service History",04/2008,331 kW,450 hp,Manual,...,13.4 l/100 km (comb)\n19.7 l/100 km (city)\n9....,None,None,None,None,None,None,None,None,None
1,1,https://www.autoscout24.com//offers/volkswagen...,"Volkswagen T4 California Exclusive 2,5TDi *Ori...","€ 31,850.-","96,300 km","Used, Full Service History",06/2000,75 kW,102 hp,Manual,...,None,06/2020,2020135279,weiß,None,None,None,None,None,None
2,2,https://www.autoscout24.com//offers/mazda-cx-5...,Mazda CX-5 G194 AWD Revolution Top SD Leder we...,"€ 31,490.-","16,100 km",Used,09/2017,143 kW,194 hp,Manual,...,7.2 l/100 km (comb)\n9.3 l/100 km (city)\n6 l/...,None,3376698,grau,,160 g CO2/km (comb),Euro 6,None,None,None
3,3,https://www.autoscout24.com//offers/fiat-tipo-...,"Fiat Tipo 1,4 95 Easy","€ 12,760.-","16,735 km","Used, Full Service History",01/2018,70 kW,95 hp,Manual,...,None,None,3434874,Schwarz,,None,Euro 6,None,None,None
4,4,https://www.autoscout24.com//offers/audi-a4-av...,"Audi A4 Avant 1,8 T FSI","€ 7,990.-","147,157 km","Used, Full Service History",07/2008,118 kW,160 hp,Manual,...,None,07/2020,None,None,None,None,None,None,None,None
5,5,https://www.autoscout24.com//offers/bmw-x5-3-0...,BMW X5 3.0sd,"€ 14,300.-","190,000 km","Used, Full Service History",06/2008,210 kW,286 hp,Manual,...,8.2 l/100 km (comb)\n10.3 l/100 km (city)\n7 l...,None,None,None,None,216 g CO2/km (comb),Euro 4,None,None,None
6,6,https://www.autoscout24.com//offers/ford-focus...,"Ford Focus Traveller 2,0 EcoBlue SCR ST-Line A...","€ 31,270.-",0 km,New,-/-,110 kW,150 hp,Manual,...,4.4 l/100 km (comb)\n5.3 l/100 km (city)\n3.9 ...,None,None,None,None,116 g CO2/km (comb),Euro 6d-TEMP,None,None,None
7,7,https://www.autoscout24.com//offers/ford-focus...,"Ford Focus Traveller 1,0 EcoBoost Trend","€ 15,990.-","14,240 km",Used,05/2019,74 kW,101 hp,Manual,...,4.7 l/100 km (comb)\n\n,None,3420902,iridium-schwarz,,107 g CO2/km (comb),Euro 6,None,None,None
8,8,https://www.autoscout24.com//offers/mercedes-b...,"Mercedes-Benz GLC 220 d 4MATIC Aut. AMG Line, ...","€ 62,900.-","1,000 km",Used,09/2019,125 kW,170 hp,Manual,...,6 l/100 km (comb)\n6.8 l/100 km (city)\n5.5 l/...,None,060021950272,obsidianschwarz metallic,None,158 g CO2/km (comb),None,None,None,None
9,9,https://www.autoscout24.com//offers/volkswagen...,Volkswagen Polo Trendline 1.0 TGI 66kW(90PS),"€ 16,850.-",10 km,New,-/-,66 kW,90 hp,Manual,...,4.8 kg/100 km (comb)\n5.5 kg/100 km (city)\n3....,None,226030,wählbar,,97 g CO2/km (comb),Euro 6d-TEMP,0603/CDS,4 (Green),None


We were interested what is the most frequent colour jkdjfkjskfjdksjf

jkdjfkjskfjdfjskdjfksjf

In [19]:
info1['Price']

0      € 9,900.-
1     € 31,850.-
2     € 31,490.-
3     € 12,760.-
4      € 7,990.-
5     € 14,300.-
6     € 31,270.-
7     € 15,990.-
8     € 62,900.-
9     € 16,850.-
10    € 15,228.-
11    € 31,690.-
12    € 21,780.-
13    € 13,790.-
14    € 27,990.-
15     € 6,500.-
16    € 12,450.-
17    € 12,570.-
18    € 18,760.-
Name: Price, dtype: object

In [22]:
mean(info1['Price'])

TypeError: can't convert type 'str' to numerator/denominator

In [21]:
from statistics import mean 